<a href="https://colab.research.google.com/github/kulkarnisatishp/genai-course/blob/main/RAG_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_key')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/GenAI/RAG/RAG with OpenAI

/content/drive/MyDrive/GenAI/RAG/RAG with OpenAI


In [4]:
!pip install pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (1,939 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
from pdf2image import convert_from_path
import os

In [6]:
def pdf_to_images(pdf_path, output_folder):
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)

  images = convert_from_path(pdf_path)
  image_paths = []
  for i, image in enumerate(images):
    image_path = os.path.join(output_folder, f"page_{i+1}.jpg")
    image.save(image_path, "JPEG")
    image_paths.append(image_path)

  return image_paths


In [7]:
pdf_path = "Things mother used to make.pdf"
output_folder = "images"
image_paths = pdf_to_images(pdf_path, output_folder)

In [8]:
!pip install openai

In [9]:
from openai import OpenAI
import base64

In [10]:
client = OpenAI(api_key=openai_api_key)
model = "gpt-4o-mini"

In [11]:
image_path = "images/page_23.jpg"
with open(image_path, "rb") as f:
  image_data = base64.b64encode(f.read()).decode('utf-8')
image_data

'...'

In [12]:
system_prompt = """
Please analyze the contents of the image and extract any recipe information
"""

In [13]:
response = client.chat.completions.create (
    model = model,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": [
            "This is the image from the recipe page.",
            { "type" : "image_url",
            "image_url" : {"url": f"data:image/jpeg;base64, {image_data}"}
              }
            ]
         }
      ]
  )


In [14]:
gpt_response = response.choices[0].message.content
print(gpt_response)

Here are the recipes extracted from the image:

### Bannocks
**Ingredients:**
- 1 Cupful of Thick Sour Milk
- ½ Cupful of Sugar
- 1 Egg
- 2 Cupfuls of Flour
- ½ Cupful of Indian Meal
- 1 Teaspoonful of Soda
- A pinch of Salt

**Instructions:**
Make the mixture stiff enough to drop from a spoon. Drop mixture, size of a walnut, into boiling fat. Serve warm, with maple syrup.

---

### Boston Brown Bread
**Ingredients:**
- 1 Cupful of Rye Meal
- 1 Cupful of Graham Meal
- ½ Cupful of Flour
- 1 Cupful of Indian Meal
- 1 Cupful of Sweet Milk
- 1 Cupful of Sour Milk
- 1 Cupful of Molasses
- ½ Teaspoonful of Salt
- 1 Heaping Teaspoonful of Soda

**Instructions:**
Stir the meals and salt together. Beat the soda into the molasses until it foams; add sour milk, mix all together and pour into a tin pail which has been well greased, if you have no brown-bread steamer.


In [15]:
from IPython.display import display, Markdown
def get_gpt_response():
  gpt_response = response.choices[0].message.content
  return display(Markdown(gpt_response))

In [18]:
system_prompt2 = """
Please analyze the contents of the image and extract any recipe information into structured component. Specifically, extract the recipe title, list of ingredients, step by step instructions, and any special instructions, cuisine type, dish type, any relevant tags or metadata
The output must be formatted in a way suitable for embedding in a retrieval augmented generation """

In [19]:
response = client.chat.completions.create (
    model = model,
    messages = [
        {"role": "system", "content": system_prompt2},
        {"role": "user", "content": [
            "This is the image from the recipe page.",
            { "type" : "image_url",
            "image_url" : {"url": f"data:image/jpeg;base64, {image_data}"}
              }
            ]
         }
      ]
  )

In [20]:
get_gpt_response()

Here’s the structured information extracted from the recipe image:

---

### Recipe Title: Bannocks

**Ingredients:**
- 1 Cupful of Thick Sour Milk
- ½ Cupful of Sugar
- 1 Egg
- 2 Cupfuls of Flour
- ½ Cupful of Indian Meal
- 1 Teaspoonful of Soda
- A pinch of Salt

**Instructions:**
1. Make the mixture stiff enough to drop from a spoon.
2. Drop mixture, size of a walnut, into boiling fat.
3. Serve warm, with maple syrup.

---

### Recipe Title: Boston Brown Bread

**Ingredients:**
- 1 Cupful of Rye Meal
- 1 Cupful of Graham Meal
- ½ Cupful of Flour
- 1 Cupful of Indian Meal
- 1 Cupful of Sweet Milk
- 1 Cupful of Sour Milk
- 1 Cupful of Molasses
- ½ Teaspoonful of Salt
- 1 Heaping Teaspoonful of Soda

**Instructions:**
1. Stir the meals and salt together.
2. Beat the soda into the molasses until it foams.
3. Add sour milk, mix all together.
4. Pour into a tin pail which has been well greased, if you have no brown-bread steamer.

---

**Cuisine Type:** Traditional/Heritage

**Dish Type:** Breads

**Tags:** Bannocks, Boston Brown Bread, Homemade, Traditional Recipes

---

In [21]:
extracted_recipes = []
for image_path in image_paths:
  with open(image_path, "rb") as f:
    image_data = base64.b64encode(f.read()).decode('utf-8')
    response = client.chat.completions.create (
      model = model,
      messages = [
          {"role": "system", "content": system_prompt2},
          {"role": "user", "content": [
              "This is the image from the recipe page.",
              { "type" : "image_url",
              "image_url" : {"url": f"data:image/jpeg;base64, {image_data}"}
                }
              ]
          }
        ],
        temperature=0
        )
    gpt_response = response.choices[0].message.content
    extracted_recipes.append({"image_path": image_path,"recipe_info":gpt_response})
    print(f"Extracted information for {image_path}\n{gpt_response}\n")

Streaming output truncated to the last 5000 lines.
- Whites of 1 Egg
- ½ Cupful of Milk
- 1½ Teaspoonful of Cream of Tartar
- ¼ Teaspoonful of Soda
- 1¾ Cupfuls of Flour
- Flavoring (amount not specified)

### Instructions
1. Cream butter and sugar together.
2. Add the well-beaten eggs, milk, flavoring, and flour into which the cream of tartar and soda have been stirred.
3. Bake for thirty minutes in a moderate oven.

### Special Instructions
- Ensure the eggs are well-beaten before adding.
- Mix thoroughly before baking.

### Cuisine Type
Traditional American

### Dish Type
Cake

### Relevant Tags
- Cake
- Baking
- Traditional Recipes

Extracted information for images/page_38.jpg
Here’s the structured information extracted from the recipe image:

### Recipe Title
- Hermits

### Ingredients
- 1 Cupful of Sugar
- ½ Cupful of Molasses
- ⅔ Cupful of Butter
- 2 Eggs
- 1 Cupful of Raisins, Chopped Fine
- 2 Tablespoonfuls of Milk
- 1 Teaspoonful of Soda
- 1 Teaspoonful of Cinnamon
- 1 Teaspo

In [22]:
filtered_recipes = []
for recipe in extracted_recipes:
  if any(keyword in recipe["recipe_info"].lower() for keyword in ["ingredients", "instructions", "recipe title"]):
    filtered_recipes.append(recipe)
  else: print(f"Skipping {recipe['image_path']}")

Skipping images/page_4.jpg
Skipping images/page_10.jpg
Skipping images/page_12.jpg
Skipping images/page_19.jpg
Skipping images/page_20.jpg
Skipping images/page_22.jpg
Skipping images/page_106.jpg
Skipping images/page_107.jpg
Skipping images/page_108.jpg
Skipping images/page_133.jpg
Skipping images/page_134.jpg
Skipping images/page_135.jpg
Skipping images/page_136.jpg


In [23]:
import json
output_file = "recipe_info.json"
with open(output_file, "w") as f:
  json.dump(filtered_recipes, f, indent=4)